In [ ]:
!nvcc -V
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!pip install -U torch==1.7.1+cu110 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'mmdetection'...
remote: Enumerating objects: 24992, done.
remote: Total 24992 (delta 0), reused 0 (delta 0), pack-reused 24992
Receiving objects: 100% (24992/24992), 37.78 MiB | 26.86 MiB/s, done.
Resolving deltas: 100% (17510/17510), done.
/content/mmdetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmdetection
  Running setup.py develop for mmdet
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |████████████████▎               | 834.1 MB 1.3 MB/s eta 0:10:26tcmalloc: large alloc 1147494400 bytes == 0x653e0000 @  0x7f4ee5155615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████▋           | 1055.7 MB 1.3 MB/s eta 0:07:33tcmalloc: large alloc 1434370048 bytes == 0x2c08000 @  0x7f4ee5155615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |██████████████████████████▏     | 1

In [ ]:
import os
import glob
import cv2
import shutil
import random
import os.path as osp
import json
import mmcv
import re
import xml.etree.ElementTree as ET
from typing import Dict, List
from mmcv.runner import init_detector
from mmcv.runner import inference_detector
from mmcv.runner import show_result_pyplot
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmcv.runner import train_detector

In [ ]:
import torch
import torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet as mmdet
print(mmdet.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"elyeskhechine","key":"87561ea6d79506ce193b2f93ee401f41"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d kbhartiya83/swimming-pool-and-car-detection

 97% 108M/111M [00:01<00:00, 146MB/s] 
100% 111M/111M [00:01<00:00, 110MB/s]


In [ ]:
import os
import zipfile

local_zip = '/content/swimming-pool-and-car-detection.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/data')
zip_ref.close()

In [ ]:
!mkdir kaggle/working
!mkdir kaggle/working/data
!mkdir kaggle/working/data/train-xml-labels
!cp -a /content/data/training_data/training_data/labels/. kaggle/working/data/train-xml-labels
!mkdir kaggle/working/data/train-images
!cp -a /content/data/training_data/training_data/images/. kaggle/working/data/train-images
!mkdir kaggle/working/data/test-images
!cp -a /content/data/test_data_images/test_data_images/images/. kaggle/working/data/test-images

mkdir: cannot create directory ‘kaggle/working’: File exists
mkdir: cannot create directory ‘kaggle/working/data’: File exists
mkdir: cannot create directory ‘kaggle/working/data/train-xml-labels’: File exists


In [ ]:
random.seed(0)
val_ids = random.sample(range(0, 3747), 200)

In [ ]:
real_val_ids = []
for ids in val_ids:
    ids = str(ids)
    length = len(ids)
    zeros_needed = 9-int(length)
    zeros = ''
    for i in range(zeros_needed):
        zeros += '0'
    zeros += ids
    real_val_ids.append(zeros)

In [ ]:
!mkdir kaggle/working/data/val-xml-labels
!mkdir kaggle/working/data/val-images
for ids in real_val_ids:
    shutil.move('kaggle/working/data/train-xml-labels/' + ids + '.xml', 'kaggle/working/data/val-xml-labels')
    shutil.move('kaggle/working/data/train-images/' + ids + '.jpg', 'kaggle/working/data/val-images')  

In [ ]:
%%writefile labels.txt
1
2

Writing labels.txt


In [ ]:
f = open("train.txt", "x")
f.close()
lines = []
for file in os.listdir('kaggle/working/data/train-xml-labels'):
    lines.append('kaggle/working/data/train-xml-labels/' + file)
with open('train.txt', 'w') as f:
    for i in range(250):
        f.write(lines[i])
        f.write('\n')
f.close()

In [ ]:
f = open("val.txt", "x")
f.close()
lines = []
for file in os.listdir('kaggle/working/data/val-xml-labels'):
    lines.append('kaggle/working/data/val-xml-labels/' + file)
with open('val.txt', 'w') as f:
    for i in range(30):
        f.write(lines[i])
        f.write('\n')
f.close()

In [ ]:
#Stolen from https://github.com/yukkyo/voc2coco
def get_label2id(labels_path: str) -> Dict[str, int]:
    with open(labels_path, 'r') as f:
        labels_str = f.read().split()
    labels_ids = list(range(1, len(labels_str)+1))
    return dict(zip(labels_str, labels_ids))


def get_annpaths(ann_dir_path: str = None,
                 ann_ids_path: str = None,
                 ext: str = '',
                 annpaths_list_path: str = None) -> List[str]:
    # If use annotation paths list
    if annpaths_list_path is not None:
        with open(annpaths_list_path, 'r') as f:
            ann_paths = f.read().split()
        return ann_paths

    # If use annotaion ids list
    ext_with_dot = '.' + ext if ext != '' else ''
    with open(ann_ids_path, 'r') as f:
        ann_ids = f.read().split()
    ann_paths = [os.path.join(ann_dir_path, aid+ext_with_dot) for aid in ann_ids]
    return ann_paths


def get_image_info(annotation_root, extract_num_from_imgid=True, train_or_val='train'):
    filename = annotation_root.findtext('filename')
    if train_or_val == 'train':
        filename = '/kaggle/working/data/train-images/' + filename
    else:
        filename = '/kaggle/working/data/val-images/' + filename 
    img_name = os.path.basename(filename)
    img_id = os.path.splitext(img_name)[0]
    if extract_num_from_imgid and isinstance(img_id, str):
        img_id = int(re.findall(r'\d+', img_id)[0])

#     size = annotation_root.find('size')
#     width = int(size.findtext('width'))
#     height = int(size.findtext('height'))
    width, height, depth = cv2.imread(filename).shape

    image_info = {
        'id': img_id,
        'width': width,
        'height': height,
        'file_name': filename,
    }
    return image_info


def get_coco_annotation_from_obj(obj, label2id):
    label = obj.findtext('name')
#     assert label in label2id, f"Error: {label} is not in label2id !"
    category_id = label2id[label]
    bndbox = obj.find('bndbox')
    xmin = int(float(bndbox.findtext('xmin')))
    ymin = int(float(bndbox.findtext('ymin')))
    xmax = int(float(bndbox.findtext('xmax')))
    ymax = int(float(bndbox.findtext('ymax')))
    assert xmax >= xmin and ymax >= ymin, f"Box size error !: (xmin, ymin, xmax, ymax): {xmin, ymin, xmax, ymax}"
    o_width = xmax - xmin
    o_height = ymax - ymin
    ann = {
        'category_id': category_id,
        'segmentation': [],  # This script is not for segmentation
        'area': o_width * o_height,
        'bbox': [xmin, ymin, o_width, o_height],
        'iscrowd': 0,
    }
    return ann


def convert_xmls_to_cocojson(annotation_paths: List[str],label2id: Dict[str, int], output_jsonpath: str, train_or_val: str, extract_num_from_imgid: bool = True):
    output_json_dict = { 
        "images": [],
        "annotations": [],
        "categories": []
    }
    bnd_id = 1  # START_BOUNDING_BOX_ID, TODO input as args ?
    print('Start converting !')
    for a_path in annotation_paths:
        # Read annotation xml
        ann_tree = ET.parse(a_path)
        ann_root = ann_tree.getroot()

        img_info = get_image_info(annotation_root=ann_root,extract_num_from_imgid=extract_num_from_imgid, train_or_val=train_or_val)
        img_id = img_info['id']
        output_json_dict['images'].append(img_info)

        for obj in ann_root.findall('object'):
            ann = get_coco_annotation_from_obj(obj=obj, label2id=label2id)
            annot = {'id': bnd_id, 'image_id': img_id,}
            annot.update(ann)
            output_json_dict['annotations'].append(annot)
            bnd_id = bnd_id + 1

    for label, label_id in label2id.items():
        category_info = {'id': label_id, 'name': label, 'supercategory': 'none'}
        output_json_dict['categories'].append(category_info)

    with open(output_jsonpath, 'w') as f:
        output_json = json.dumps(output_json_dict)
        f.write(output_json)

In [ ]:
def convert_to_coco(ann_path_list='/content/train.txt', labels='/content/labels.txt', output='/content/output.json', train_or_val='train'):
    label2id = get_label2id(labels_path=labels)
    ann_paths = get_annpaths(
        annpaths_list_path=ann_path_list
    )
    convert_xmls_to_cocojson(
        annotation_paths=ann_paths,
        label2id=label2id,
        output_jsonpath=output,
        train_or_val=train_or_val
    )

In [ ]:
convert_to_coco()

In [ ]:
convert_to_coco(ann_path_list='/content/val.txt', labels='/content/labels.txt', output='/content/val_output.json', train_or_val='val')

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_1x_coco.py')

In [ ]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'CocoDataset'
cfg.classes = '/kaggle/working/labels.txt'
cfg.data_root = '/kaggle/working'

cfg.model.roi_head.bbox_head.num_classes = 2

cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = '/kaggle/working'
cfg.data.test.ann_file = 'val_output.json'
cfg.data.test.img_prefix = 'data/val-images'

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = '/kaggle/working'
cfg.data.train.ann_file = 'output.json'
cfg.data.train.img_prefix = 'data/train-images'
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = '/kaggle/working'
cfg.data.val.ann_file = 'val_output.json'
cfg.data.val.img_prefix = 'data/val-images'
cfg.data.val.classes = 'labels.txt'

cfg.load_from = '../input/faster-rcnn-checkpoint/faster_rcnn_x101_64x4d_fpn_1x_coco_20200204-833ee192.pth'

cfg.work_dir = '/kaggle/working/model_output'

cfg.optimizer.lr = 0.005
cfg.lr_config.step = 5
cfg.optimizer.weight_decay = 0.005
# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.lr_config.warmup_iters = 250
cfg.lr_config.policy = 'step'
# cfg.lr_config.warmup = none
# cfg.lr_config.warmup_ratio = 0.1
# cfg.lr_config.min_lr_ratio = 1e-5
# cfg.lr_config = dict(warmup_iters=1000, policy='CosineAnnealing', warmup='linear', warmup_ratio=0.1, min_lr_ratio=1e-6)
cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 1
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 4
cfg.checkpoint_config.interval = 6
cfg.runner.max_epochs = 18
cfg.log_config.interval = 250

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

print(f'Config:\n{cfg.pretty_text}')

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(
 cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
model = init_detector(cfg, '/kaggle/working/model_output/epoch_18.pth')
for file in os.listdir('/kaggle/working/data/val-images'):
    img = mmcv.imread('/kaggle/working/data/val-images/' + file)
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result)